In [2]:
import re
import string
import pandas as pd

import nltk
from nltk.corpus import stopwords
# Download Spanish stopwords if not already downloaded
nltk.download('stopwords')

import spacy
import es_core_news_sm
nlp = es_core_news_sm.load()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\garim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\garim\anaconda3\envs\nlp\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
spanish_stopwords = set(stopwords.words('spanish'))

def remove_stopwords(text, stop_words= spanish_stopwords):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

def lemmatize_words(words):
    doc = nlp(' '.join(words))
    return [token.lemma_ for token in doc]

def preprocess_text(text, remove_stopwords= True, remove_stopwords_func= remove_stopwords):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    # text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()

    if remove_stopwords:
        text = remove_stopwords_func(text)
        
    return text


In [29]:
# Example usage
spanish_text = "¡Hola! Visitas https://example.com para más información. ¿Cuántos años tienes? Tengo 25 años."
cleaned_text = preprocess_text(spanish_text, remove_stopwords= False)
print(cleaned_text)

cleaned_text = remove_stopwords(cleaned_text)
print(cleaned_text)

cleaned_text = " ".join(lemmatize_words(cleaned_text.split()))
print(cleaned_text)

hola visitas para más información cuántos años tienes tengo años
hola visitas información cuántos años años
holar visita información cuántos año año


In [5]:
df = pd.read_csv('data/recetas_limpias.csv')
df.head()

,nombre,url,ingredientes,pasos,pais,duracion,porciones,calorias,categoria,contexto,comensales,tiempo,dificultad,categoria 2,valoracion,votos
0,berenjenas rellenas,https://www.elmueble.com/cocinas/comidas-salud...,"2 berenjenas, 1 pimiento rojo, 1 pimiento amar...",paso 1. lava bien las berenjenas y pártelas po...,españa,45 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"alto en fibra, sin grasas trans, sin sodio o s...",NaN,NaN
1,alcachofas al horno con pico de gallo,https://www.elmueble.com/cocinas/comidas-salud...,"4 alcachofas, 1 limón, 400 g de carne picada (...","paso 1. precalentar el horno a 180?°c. , paso ...",españa,60 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, sin grasa, alto en fibra",NaN,NaN
2,arroz basmati salteado con heura y verduras va...,https://www.elmueble.com/cocinas/comidas-salud...,"200 g de arroz basmati integral, 450 ml de agu...",paso 1. hervir el arroz basmati durante unos 2...,españa,40 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, alto en grasas, bueno fuente...",NaN,NaN
3,tataki de atún,https://www.elmueble.com/cocinas/comidas-salud...,"200 gramos de atún rojo (fresco o congelado), ...",paso 1. cortar en forma rectangular para el ta...,españa,60 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, sin grasa, alto en fibra",NaN,NaN
4,merluza al vapor a la gallega,https://www.elmueble.com/cocinas/comidas-salud...,"700 g de merluza en rodajas, 3 ajos, 1 manojo ...",paso 1. pelar y lavar las patatas. retirar el ...,españa,45 min,4,NaN,NaN,NaN,NaN,NaN,NaN,"bajo en calorías, sin grasa, alto en fibra",NaN,NaN


In [37]:
from gensim.models import Word2Vec

# Preprocess the 'pasos' column
train_data = df['pasos'].dropna().apply(preprocess_text).str.split()
train_data = pd.concat([train_data, df['contexto'].dropna().apply(remove_stopwords).str.split()], axis=0) # add context sentences
train_data = pd.concat([train_data, df['ingredientes'].dropna().apply(remove_stopwords).str.split()], axis=0) # add inhredient list

# Train Word2Vec model
w2v_model = Word2Vec(sentences= train_data, vector_size=25, window=7, min_count= 10, workers=4)

In [38]:
w2v_model.wv.most_similar('azucar', topn=10)

[('azãºcar', 0.8528393507003784),
 ('azúcar', 0.8387829661369324),
 ('canela', 0.7782072424888611),
 ('aguardiente', 0.756539523601532),
 ('esencia', 0.7510204911231995),
 ('vainilla', 0.7410455942153931),
 ('licor', 0.7176855802536011),
 ('ron', 0.7173025012016296),
 ('(para', 0.7147694826126099),
 ('colapez', 0.7064535617828369)]